In [1]:
import sys
import cv2 as cv
import numpy as np
import os

sys.path.append('/home/sardarchitect/repos/github.com/ucsd_capstone/')
from streetstudy.data import virat

In [2]:
videos_df = virat.build()
current_video = videos_df.iloc[2]
annotations_df = virat.get_annotations(current_video.path)

In [3]:
def convert_virat_to_yolo(video, df):
    df = df.copy()
    capture = cv.VideoCapture(video.path)
    image_width = capture.get(cv.CAP_PROP_FRAME_WIDTH)
    image_height = capture.get(cv.CAP_PROP_FRAME_HEIGHT)
    image_channels = capture.get(cv.CAP_PROP_FRAME_WIDTH)
    
    df['object_type'] = 0
    df['bbox_center_x'] = df['bbox_lefttop_x'] + (df['bbox_width'] / 2)
    df['bbox_center_y'] = df['bbox_lefttop_y'] + (df['bbox_height'] / 2)
   
    # Normalize
    df['bbox_center_x'] = round(df['bbox_center_x'] / image_width, 3)
    df['bbox_center_y'] = round(df['bbox_center_y'] / image_height, 3)
    df['bbox_width'] = (df['bbox_width'] / image_width)
    df['bbox_height'] = (df['bbox_height'] / image_height)
    
    for current_frame in (df['current_frame'].unique()):
        bboxs = df[df['current_frame'] == current_frame][['object_type', 'bbox_center_x', 'bbox_center_y', 'bbox_width', 'bbox_height']]
        if not os.path.exists(f'./.data_cache/{video.name}'):
            os.mkdir(f'./.data_cache/{video.name}')
        np.savetxt(f'./.data_cache/{video.name}/{current_frame}.txt', bboxs.values, fmt='%.4f')

In [4]:
convert_virat_to_yolo(current_video, annotations_df)